### Data Processing Investigation

Load spreadsheet - multiple sheets, multiple tables

In [16]:
import pandas as pd
import numpy as np

# Data source:
# Car driving test cancellation reasons by test centre
# https://www.gov.uk/government/statistical-data-sets/car-driving-test-data-by-test-centre
# 1.13 MB as of 6th May 2022
# Last updated: 

FILE_PATH = r'C:\Users\lnick\Documents\Personal\Programming\Python\Data Science\dvsa1205.ods'
SHEET_NAMES = [f'20{x}-{x + 1}' for x in range(21, 9, -1)]

dfs = pd.read_excel(FILE_PATH, sheet_name=None, engine='odf')  # dict - {sheet_name: DataFrame}

for sheet_name in dfs.copy():                       # delete unwanted sheets
    if sheet_name not in SHEET_NAMES:
        dfs.pop(sheet_name)

Preliminary cleaning - extract only the regions with the data

In [17]:
for sheet_name, df in dfs.items():
    df.drop([0, 1, 2, 3], inplace=True)             # remove first 4 rows (no data)
    df.dropna(axis=1, how='all', inplace=True)      # remove columns which are all null
    df.iloc[1, -1] = df.iloc[0, -1]                 # copy merged cell to correct row below
    df.reset_index(drop='ffill', inplace=True)      # unmerge cell
    df.rename(columns=df.iloc[1], inplace=True)     # copy correct table headers
    df.drop([0, 1], inplace=True)                   # remove pointless rows
    df.dropna(axis=0, how='all', inplace=True)      # remove rows which are all null
    df.reset_index(drop=True, inplace=True)         # reset table row indices

Obtain a list of all locations and column names

In [18]:
col_1 = pd.concat([df.iloc[:, 0] for df in dfs.values()], axis=0)   # combine all first columns to a Series
col_1.drop_duplicates(inplace=True)                                 # remove all duplicate rows
col_df = pd.DataFrame(col_1, columns=['entry'])                     # convert to DataFrame with one column
col_df.dropna(inplace=True)                                         # remove all na's
col_df = col_df[col_df['entry'].map(                                # remove all entries with a year (not a place)
    lambda x: not ('20' in str(x) or x == 'National'))]             # or 'National' to avoid edge case in counting
col_df.reset_index(inplace=True)                                    # reset row index
place_names = pd.Series(col_df['entry'])                            # convert back to Series

col_names = pd.concat([
    pd.Series(df.columns) for df in dfs.values()])                  # combine column names of all sheets
col_names = pd.Series(
    col_names.dropna().drop_duplicates().reset_index(drop=True))    # clean and convert to Series

Obtain data for each place

In [33]:
dataset = {}
MONTHS = ['January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December']

for place in place_names:
    df_new = pd.DataFrame(columns=df.columns)
    for sheet_name, df in dfs.items():
        indices = df.index[df.iloc[:, 0] == place].tolist()                         # search for place name index
        if indices != []:
            partial_df_new = df.iloc[indices[0] + 1 : indices[1]]                   # extract the appropriate range

        df_new = pd.concat([df_new, partial_df_new], sort=False, join='outer')      # add it to the record for this place
    
    # clean up
    df_new.dropna(axis=1, how='all', inplace=True)                                  # first column is all na's, remove it
    df_new.fillna(0, inplace=True)                                                  # replace missing entries with 0
    df_new.reset_index(drop=True, inplace=True)                                     # count rows from 0
    df_new.rename(columns={df.columns[0]: 'Month'}, inplace=True)                   # set name of first column
    df_new['Month'] = pd.to_datetime(df_new['Month'], format='%B %Y')               # convert month/year names to datetime  # NOTE: before: errors='coerce'
    df_new.sort_values(by='Month', ascending=False, ignore_index=True, inplace=True)# sort from most recent to oldest
    df_new.loc['Totals']= df_new.iloc[:, 1:].sum()                                  # get column (group) totals
    df_new = df_new.convert_dtypes()                                                # NOTE: 'sum' of date column given
                                                                                    # as pd.NaT, hard to change due to:
                                                                                    # https://github.com/pandas-dev/pandas/issues/29024#issuecomment-1098052276
    # swap 'pandemic' and row totals columns
    if 'Pandemic' in list(df_new.columns):
        old_pandemic_column = df_new['Pandemic']
        df_new.drop('Pandemic', axis=1, inplace=True)
        df_new.insert(len(df_new.columns) - 1, 'Pandemic', old_pandemic_column)

    dataset.update({place: df_new})                                                 # record the data at this place key


In [36]:
dataset['Aberdeen North']

,Month,Leave,Disputes,Acts of Nature,Medical Absences,Pandemic,Total Tests Cancelled
0,2021-12-01,0,0,46,1,0,47
1,2021-11-01,0,0,8,0,0,8
2,2021-10-01,0,0,0,25,17,42
3,2021-08-01,68,0,0,11,0,79
4,2021-05-01,25,0,0,0,1,26
...,...,...,...,...,...,...,...
81,2015-01-01,0,0,1,0,0,1
82,2015-01-01,0,0,1,0,0,1
83,2015-01-01,0,0,1,0,0,1
84,2015-01-01,0,0,1,0,0,1
